# Redes neurais Recorrentes (RNN)

### Bibliotecas básicas

In [1]:
import numpy as np
import pandas as pd
from functools import partial
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
print(tf.__version__)

2.1.0


In [2]:
tf.compat.v1.disable_eager_execution()

### RNN básica - implementação manual

In [3]:
tf.compat.v1.reset_default_graph()

n_inputs = 3
n_neurons = 5

X0 = tf.compat.v1.placeholder(tf.float32, [None, n_inputs])
X1 = tf.compat.v1.placeholder(tf.float32, [None, n_inputs])

Wx = tf.Variable(tf.compat.v1.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))
Wy = tf.Variable(tf.compat.v1.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))

Y0 = tf.math.tanh(tf.matmul(X0, Wx) + b)
Y1 = tf.math.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b)

init = tf.compat.v1.global_variables_initializer()

X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]]) # t = 0
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]]) # t = 1

with tf.compat.v1.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


#### Saída da rede em ambos intervalos para todos os neurônios e todas as instâncias no minilote:

In [4]:
print(Y0_val)

[[ 0.08133698  0.9461138   0.9992334  -0.5767777  -0.99837404]
 [-0.99998575 -0.93964344  1.         -0.9999966  -1.        ]
 [-1.         -0.9999464   1.         -1.         -1.        ]
 [-1.         -1.          0.99999994 -1.         -1.        ]]


In [5]:
print(Y1_val)

[[-1.         -1.          1.         -1.         -1.        ]
 [-0.758769   -0.95284617 -0.31331128  0.65702546 -0.9547951 ]
 [-1.         -1.          1.         -1.         -1.        ]
 [-0.99999833 -0.9999989   0.9999974  -0.99975085 -0.9999885 ]]


### Desenrolamento estático através do tempo

In [6]:
tf.compat.v1.reset_default_graph()

n_inputs = 3
n_neurons = 5

X0 = tf.compat.v1.placeholder(tf.float32, [None, n_inputs])
X1 = tf.compat.v1.placeholder(tf.float32, [None, n_inputs])

basic_cell = tf.compat.v1.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.compat.v1.nn.static_rnn(basic_cell, [X0, X1], dtype=tf.float32)
Y0, Y1 = output_seqs

init = tf.compat.v1.global_variables_initializer()

X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]])
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]])

with tf.compat.v1.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
print(Y0_val)

[[ 0.5602356  -0.151552   -0.13176677 -0.23124242  0.7767599 ]
 [-0.58441734 -0.94278073 -0.31290483 -0.63624394  0.99970096]
 [-0.96195525 -0.9976481  -0.47382185 -0.8533018   0.9999997 ]
 [-0.99984044 -0.9999935  -0.39269054 -0.7114957   0.99999505]]


In [8]:
print(Y1_val)

[[-0.9992447  -0.99997085 -0.79857796 -0.8924509   1.        ]
 [ 0.1243832  -0.5071309  -0.6011743   0.26781803  0.90067124]
 [-0.99676526 -0.998416   -0.6972026  -0.23852691  0.99999946]
 [-0.9585109  -0.9515962  -0.5853359   0.17232035  0.99946636]]


### Desenrolamento dinâmico através do tempo

In [9]:
n_steps = 2
n_inputs = 3
n_neurons = 5

tf.compat.v1.reset_default_graph()

X = tf.compat.v1.placeholder(tf.float32, [None, n_steps, n_inputs])

basic_cell = tf.compat.v1.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.compat.v1.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

init = tf.compat.v1.global_variables_initializer()

X_batch = np.array([
        [[0, 1, 2], [9, 8, 7]], # instância 1
        [[3, 4, 5], [0, 0, 0]], # instância 2
        [[6, 7, 8], [6, 5, 4]], # instância 3
        [[9, 0, 1], [3, 2, 1]], # instância 4
    ])

with tf.compat.v1.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X: X_batch})

print(outputs_val)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
[[[ 0.8737009  -0.695371   -0.16444561 -0.5569858  -0.00353421]
  [ 0.9999996  -0.99998724 -0.84644794 -0.9953022  -0.992788  ]]

 [[ 0.9995603  -0.995603   -0.70460784 -0.9454561  -0.8067972 ]
  [ 0.27718204 -0.21497945  0.8087338  -0.4887352   0.4198085 ]]

 [[ 0.9999985  -0.9999461  -0.9196625  -0.99448997 -0.97723144]
  [ 0.9998222  -0.9989662  -0.1431439  -0.9796231  -0.95658356]]

 [[ 0.91071117 -0.9851721   0.65366745  0.9754118  -0.88200337]
  [ 0.9858268  -0.5021983   0.3248572  -0.6346006  -0.50137085]]]


### Manipulando sentenças de entrada de comprimento variável

In [10]:
n_steps = 2
n_inputs = 3
n_neurons = 5

tf.compat.v1.reset_default_graph()

X = tf.compat.v1.placeholder(tf.float32, [None, n_steps, n_inputs])
seq_length = tf.compat.v1.placeholder(tf.int32, [None])

basic_cell = tf.compat.v1.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.compat.v1.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32, sequence_length=seq_length)

init = tf.compat.v1.global_variables_initializer()

X_batch = np.array([
        #step 0     step 1
        [[0, 1, 2], [9, 8, 7]], # instância 1
        [[3, 4, 5], [0, 0, 0]], # instância 2 (preenchido com vetor de zeros)
        [[6, 7, 8], [6, 5, 4]], # instância 3
        [[9, 0, 1], [3, 2, 1]], # instância 4
    ])

seq_length_batch = np.array([2, 1, 2, 2])

with tf.compat.v1.Session() as sess:
    init.run()
    outputs_val, states_val = sess.run([outputs, states], feed_dict={X: X_batch, seq_length: seq_length_batch})

print(outputs_val)

[[[ 0.00294726  0.40077475 -0.81542796 -0.34734768  0.49147463]
  [-0.96901876 -0.99999523 -1.          0.96501315  0.9936624 ]]

 [[-0.74088514 -0.93418354 -0.9999301   0.33584136  0.9490635 ]
  [ 0.          0.          0.          0.          0.        ]]

 [[-0.9569023  -0.9990097  -1.          0.78613037  0.9960017 ]
  [-0.71336406 -0.9999559  -0.9999992   0.7000074   0.9899887 ]]

 [[-0.99721456 -0.9998737  -0.99991035  0.9902564   0.9975323 ]
  [ 0.00871231 -0.99722683 -0.9977773   0.0639837   0.883115  ]]]


### Treinando um classificador de sequência

In [11]:
tf.compat.v1.reset_default_graph()

n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10

learning_rate = 0.001

X = tf.compat.v1.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.compat.v1.placeholder(tf.int32, [None])

basic_cell = tf.compat.v1.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.compat.v1.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

logits = tf.compat.v1.layers.dense(states, n_outputs)
xentropy = tf.compat.v1.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy)

optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

correct = tf.compat.v1.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.compat.v1.global_variables_initializer()

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [12]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch
        
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

X_test = X_test.reshape((-1, n_steps, n_inputs))

In [13]:
n_epochs = 100
batch_size = 150

with tf.compat.v1.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if epoch % 10 == 0:
            acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
            acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
            print(epoch, 'Last batch accuracy:', acc_batch, 'Test accuracy:', acc_test)

0 Last batch accuracy: 0.91333336 Test accuracy: 0.9279
10 Last batch accuracy: 0.96 Test accuracy: 0.965
20 Last batch accuracy: 0.9866667 Test accuracy: 0.9775
30 Last batch accuracy: 0.9866667 Test accuracy: 0.9749
40 Last batch accuracy: 1.0 Test accuracy: 0.9752
50 Last batch accuracy: 0.99333334 Test accuracy: 0.9743
60 Last batch accuracy: 0.99333334 Test accuracy: 0.9778
70 Last batch accuracy: 0.97333336 Test accuracy: 0.9731
80 Last batch accuracy: 1.0 Test accuracy: 0.9805
90 Last batch accuracy: 0.99333334 Test accuracy: 0.9784


### Prevendo Séries temporais

In [16]:
t_min, t_max = 0, 30
resolution = 0.1

def time_series(t):
    return t * np.sin(t) / 3 + 2 * np.sin(t*5)

def next_batch(batch_size, n_steps):
    t0 = np.random.rand(batch_size, 1) * (t_max - t_min - n_steps * resolution)
    Ts = t0 + np.arange(0., n_steps + 1) * resolution
    ys = time_series(Ts)
    return ys[:, :-1].reshape(-1, n_steps, 1), ys[:, 1:].reshape(-1, n_steps, 1)

In [17]:
tf.compat.v1.reset_default_graph()

n_steps = 20
n_inputs = 1
n_neurons = 100

X = tf.compat.v1.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.compat.v1.placeholder(tf.float32, [None, n_steps, n_outputs])

cell = tf.compat.v1.nn.rnn_cell.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu)
rnn_outputs, states = tf.compat.v1.nn.dynamic_rnn(cell, X, dtype=tf.float32)

n_outputs = 1
learning_rate = 0.001

stacked_rnn_outputs = tf.reshape(rnn_outputs, [-1, n_neurons])
stacked_outputs = tf.compat.v1.layers.dense(stacked_rnn_outputs, n_outputs)
outputs = tf.reshape(stacked_outputs, [-1, n_steps, n_outputs])

loss = tf.reduce_mean(tf.square(outputs - y))
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

init = tf.compat.v1.global_variables_initializer()
saver = tf.compat.v1.train.Saver()

n_iterations = 1500
batch_size = 50
t_instance = np.linspace(12.2, 12.2 + resolution * (n_steps + 1), n_steps + 1)

with tf.compat.v1.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        X_batch, y_batch = next_batch(batch_size, n_steps)
        sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if iteration % 100 == 0:
            mse = loss.eval(feed_dict={X: X_batch, y: y_batch})
            print(iteration, '\tMSE:', mse)
    
    X_new = time_series(np.array(t_instance[:-1].reshape(-1, n_steps, n_inputs)))
    y_pred = sess.run(outputs, feed_dict={X: X_new})    
    saver.save(sess, 'model/my_time_series_model')

0 	MSE: 17.23357
100 	MSE: 0.64479566
200 	MSE: 0.22471042
300 	MSE: 0.08508409
400 	MSE: 0.06518517
500 	MSE: 0.048398558
600 	MSE: 0.049764246
700 	MSE: 0.063373424
800 	MSE: 0.051421843
900 	MSE: 0.050426837
1000 	MSE: 0.047503926
1100 	MSE: 0.05374335
1200 	MSE: 0.051288188
1300 	MSE: 0.03611956
1400 	MSE: 0.047032144


In [18]:
y_pred

array([[[-3.4835126],
        [-2.5070946],
        [-1.1347352],
        [ 0.6665767],
        [ 1.9816937],
        [ 3.0658002],
        [ 3.5624275],
        [ 3.3560722],
        [ 2.8085177],
        [ 2.1856112],
        [ 1.7481649],
        [ 1.5522208],
        [ 1.8859161],
        [ 2.7251291],
        [ 3.87829  ],
        [ 5.0895925],
        [ 6.1035843],
        [ 6.679761 ],
        [ 6.662667 ],
        [ 6.0519094]]], dtype=float32)